<a href="https://colab.research.google.com/github/kmongsil1105/colab_ipynb/blob/main/AI_PyTorch(%EB%8B%A8%EC%96%B4_%EB%8B%A8%EC%9C%84_RNN_%EC%9E%84%EB%B2%A0%EB%94%A9_%EC%82%AC%EC%9A%A9).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 단어 단위 RNN - 임베딩 사용

문자 단위가 아니라 RNN의 입력 단위를 단어 단위로 사용합니다.

단어 단위를 사용함에 따라서 Pytorch에서 제공하는 임베딩 층(embedding layer)를 사용하겠습니다.

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

실습을 위해 임의의 문장을 만듭니다.

In [ ]:
sentence = "Repeat is the best medicine for memory".split()

우리가 만들 RNN은 'Repeat is the best medicine for'을 입력받으면 'is the best medicine for memory'를 출력하는 RNN입니다.

위의 임의의 문장으로부터 단어장(vocabulary)을 만듭니다.

In [ ]:
vocab = list(set(sentence))
print(vocab)

['is', 'for', 'the', 'Repeat', 'memory', 'medicine', 'best']


이제 단어장의 단어에 고유한 정수 인덱스를 부여합니다. 그리고 그와 동시에 모르는 단어를 의미하는 UNK 토큰도 추가하겠습니다.

In [ ]:
word2index = {tkn: i for i, tkn in enumerate(vocab, 1)}  # 단어에 고유한 정수 부여
word2index['<unk>']=0

In [ ]:
print(word2index)

{'is': 1, 'for': 2, 'the': 3, 'Repeat': 4, 'memory': 5, 'medicine': 6, 'best': 7, '<unk>': 0}


word2index가 우리가 사용할 최종 단어장인 셈입니다. word2index에 단어를 입력하면 맵핑되는 정수를 리턴합니다.

In [ ]:
print(word2index['memory'])

5


단어 'memory'와 맵핑되는 정수는 2입니다. 예측 단계에서 예측한 문장을 확인하기 위해 idx2word도 만듭니다.

In [ ]:
# 수치화된 데이터를 단어로 바꾸기 위한 사전
index2word = {v: k for k, v in word2index.items()}
print(index2word)

{1: 'is', 2: 'for', 3: 'the', 4: 'Repeat', 5: 'memory', 6: 'medicine', 7: 'best', 0: '<unk>'}


idx2word는 정수로부터 단어를 리턴하는 역할을 합니다. 정수 2를 넣어봅시다.

In [ ]:
print(index2word[2])

for


==> 정수 2와 맵핑되는 단어는 memory인 것을 확인할 수 있습니다.

제 데이터의 각 단어를 정수로 인코딩하는 동시에, 입력 데이터와 레이블 데이터를 만드는 build_data라는 함수를 만들어보겠습니다.

In [ ]:
def build_data(sentence, word2index):
    encoded = [word2index[token] for token in sentence] # 각 문자를 정수로 변환. 
    input_seq, label_seq = encoded[:-1], encoded[1:] # 입력 시퀀스와 레이블 시퀀스를 분리
    input_seq = torch.LongTensor(input_seq).unsqueeze(0) # 배치 차원 추가
    label_seq = torch.LongTensor(label_seq).unsqueeze(0) # 배치 차원 추가
    return input_seq, label_seq

만들어진 함수로부터 입력 데이터와 레이블 데이터를 얻습니다.

In [ ]:
X, Y = build_data(sentence, word2index)

입력 데이터와 레이블 데이터가 정상적으로 생성되었는지 출력해봅시다.

In [ ]:
# sentence = "Repeat is the best medicine for memory".split()
print(X)   # {1: 'is', 2: 'for', 3: 'the', 4: 'Repeat', 5: 'memory', 6: 'medicine', 7: 'best', 0: '<unk>'}
print(Y)

############ 결과
#tensor([[7, 4, 3, 1, 6, 5]]) # Repeat is the best medicine for을 의미
#tensor([[4, 3, 1, 6, 5, 2]]) # is the best medicine for memory을 의미

tensor([[4, 1, 3, 7, 6, 2]])
tensor([[1, 3, 7, 6, 2, 5]])


# 모델 구현하기

==> 전 모델들과 달라진 점은 임베딩 층을 추가했다는 점입니다. 파이토치에서는 nn.Embedding()을 사용해서 임베딩 층을 구현합니다. 임베딩층은 크게 두 가지 인자를 받는데 첫번째 인자는 단어장의 크기이며, 두번째 인자는 임베딩 벡터의 차원입니다.

In [ ]:
class Net(nn.Module):
    def __init__(self, vocab_size, input_size, hidden_size, batch_first=True):
        super(Net, self).__init__()
        self.embedding_layer = nn.Embedding(num_embeddings=vocab_size, # 워드 임베딩
                                            embedding_dim=input_size)
        self.rnn_layer = nn.RNN(input_size, hidden_size, # 입력 차원, 은닉 상태의 크기 정의
                                batch_first=batch_first)
        self.linear = nn.Linear(hidden_size, vocab_size) # 출력은 원-핫 벡터의 크기를 가져야함. 또는 단어 집합의 크기만큼 가져야함.

    def forward(self, x):
        # 1. 임베딩 층
        # 크기변화: (배치 크기, 시퀀스 길이) => (배치 크기, 시퀀스 길이, 임베딩 차원)
        output = self.embedding_layer(x)
        # 2. RNN 층
        # 크기변화: (배치 크기, 시퀀스 길이, 임베딩 차원)
        # => output (배치 크기, 시퀀스 길이, 은닉층 크기), hidden (1, 배치 크기, 은닉층 크기)
        output, hidden = self.rnn_layer(output)
        # 3. 최종 출력층
        # 크기변화: (배치 크기, 시퀀스 길이, 은닉층 크기) => (배치 크기, 시퀀스 길이, 단어장 크기)
        output = self.linear(output)
        # 4. view를 통해서 배치 차원 제거
        # 크기변화: (배치 크기, 시퀀스 길이, 단어장 크기) => (배치 크기*시퀀스 길이, 단어장 크기)
        return output.view(-1, output.size(2))

이제 모델을 위해 하이퍼파라미터를 설정합니다.

In [ ]:
# 하이퍼 파라미터
vocab_size = len(word2index)  # 단어장의 크기는 임베딩 층, 최종 출력층에 사용된다. <unk> 토큰을 크기에 포함한다.
input_size = 5  # 임베딩 된 차원의 크기 및 RNN 층 입력 차원의 크기
hidden_size = 20  # RNN의 은닉층 크기

모델을 생성합니다.

In [ ]:
# 모델 생성
model = Net(vocab_size, input_size, hidden_size, batch_first=True)
# 손실함수 정의
loss_function = nn.CrossEntropyLoss() # 소프트맥스 함수 포함이며 실제값은 원-핫 인코딩 안 해도 됨.
# 옵티마이저 정의
optimizer = optim.Adam(params=model.parameters())

모델에 입력을 넣어서 출력을 확인해봅시다.

In [ ]:
# 임의로 예측해보기. 가중치는 전부 랜덤 초기화 된 상태이다.
output = model(X)
print(output)

tensor([[-0.5570,  0.0234, -0.2844, -0.1279, -0.0263, -0.3691, -0.4098, -0.1306],
        [-0.3032, -0.1547, -0.2559, -0.3821,  0.1355, -0.3250, -0.2312, -0.1613],
        [-0.2627, -0.1161, -0.2290, -0.4161,  0.1349, -0.3649, -0.1319, -0.0724],
        [-0.0477, -0.2737,  0.1242,  0.0871,  0.5134, -0.0757, -0.1776, -0.1219],
        [-0.2574, -0.0315, -0.0053, -0.2669,  0.2663, -0.2883, -0.1990,  0.0245],
        [-0.0852,  0.0686, -0.2270, -0.3748,  0.2887, -0.2412, -0.4358,  0.1846]],
       grad_fn=<ViewBackward>)


모델이 어떤 예측값을 내놓기는 하지만 현재 가중치는 랜덤 초기화되어 있어 의미있는 예측값은 아닙니다. 예측값의 크기를 확인해봅시다

In [ ]:
print(output.shape)

torch.Size([6, 8])


예측값의 크기는 (6, 8)입니다. 이는 각각 (시퀀스의 길이, 은닉층의 크기)에 해당됩니다. 모델은 훈련시키기 전에 예측을 제대로 하고 있는지 예측된 정수 시퀀스를 다시 단어 시퀀스로 바꾸는 decode 함수를 만듭니다.

In [ ]:
# 수치화된 데이터를 단어로 전환하는 함수
decode = lambda y: [index2word.get(x) for x in y]

약 200 에포크 학습합니다.

In [ ]:
# 훈련 시작
for step in range(201):
    # 경사 초기화
    optimizer.zero_grad()
    # 순방향 전파
    output = model(X)
    # 손실값 계산
    loss = loss_function(output, Y.view(-1))
    # 역방향 전파
    loss.backward()
    # 매개변수 업데이트
    optimizer.step()
    # 기록
    if step % 40 == 0:    # # sentence = "Repeat is the best medicine for memory".split()
        print("[{:02d}/201] {:.4f} ".format(step+1, loss))
        pred = output.softmax(-1).argmax(-1).tolist()
        print(" ".join(["Repeat"] + decode(pred)))
        print()

[01/201] 2.1050 
Repeat is Repeat Repeat Repeat Repeat Repeat

[41/201] 1.5303 
Repeat is the best Repeat for memory

[81/201] 0.8460 
Repeat is the best medicine for memory

[121/201] 0.3961 
Repeat is the best medicine for memory

[161/201] 0.2084 
Repeat is the best medicine for memory

[201/201] 0.1301 
Repeat is the best medicine for memory

